In [9]:
import xarray as xr
import time
st = time.time()
#xr.open_mfdataset('~/R/gebco/*.nc', parallel=True)
ds = xr.open_mfdataset(
    'GEBCO_2022_sub_ice_topo.nc', chunks={0: 60, 1: 60}, 
    decode_cf=False, decode_times=False) #.load()
et = time.time()
print('Exe time: ', et-st, 'sec')
ds

Exe time:  0.0043637752532958984 sec


<xarray.Dataset>
Dimensions:    (lon: 86400, lat: 43200)
Coordinates:
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(43200, 86400), meta=np.ndarray>
Attributes: (12/36)
    title:                           The GEBCO_2022 Grid - a continuous terra...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    node_offset:                     1.0

In [10]:
print(sys.getsizeof(ds))
ds['elevation'].nbytes/(1024 ** 3)

112


6.9522857666015625

In [33]:
print(ds.sel(lat=23.326173, lon=123.978125, method='nearest')['elevation'].values)
print(ds.sel(lat=23.317670, lon=123.973958, method='nearest')['elevation'].values)
print(ds.sel(lat=21.336378, lon=123.003125, method='nearest')['elevation'].values)
print(ds.sel(lat=21.33, lon=123.0, method='nearest')['elevation'].values)
print(ds["elevation"].values[27200,72956]) #this is correct value (got depth = -3729)
print(ds["elevation"].values[27200,72957])
print(ds["elevation"].values[27200,72955])
print(ds["elevation"].values[27201,72956])
print(ds["elevation"].values[27199,72956]) #But we got -3813, this is: after subset got a latitude offset 1? why (in zprofile01.ipynb correct it)

-3949
-4278
-4819
-4766
-3729
-3714
-3739
-3677
-3813


In [12]:
#The following try different subsetting and index methods
st = time.time()
ds_s1 = ds.sel(lon=slice(105, 135), lat=slice(2, 35)) 
ds_s1.close() #try if it can close
et = time.time()
print('Exe time: ', et-st, 'sec')

Exe time:  0.0009481906890869141 sec


In [13]:
ds_s1['elevation'].nbytes/(1024 ** 3)

0.1062154769897461

In [14]:
#da1 = getattr(ds_s1, 'elevation')
#da1
#https://www.programcreek.com/python/example/123571/xarray.decode_cf
#da1.attrs['_Unsigned']='false'
ds_s1


<xarray.Dataset>
Dimensions:    (lon: 7200, lat: 7920)
Coordinates:
  * lon        (lon) float64 105.0 105.0 105.0 105.0 ... 135.0 135.0 135.0 135.0
  * lat        (lat) float64 2.002 2.006 2.01 2.015 ... 34.99 34.99 34.99 35.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(7920, 7200), meta=np.ndarray>
Attributes: (12/36)
    title:                           The GEBCO_2022 Grid - a continuous terra...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    node_offset:                     1.0

In [ ]:
#da1

In [15]:
st = time.time()
st1 = ds_s1.isel(lon=slice(1000, 1010), lat=slice(2000, 2010))
xt1 = st1['elevation'].values
et = time.time()
print('Exe time: ', et-st, 'sec')
print(xt1)

Exe time:  0.003007650375366211 sec
[[-122 -120 -117 -113 -108 -103  -97  -92  -85  -79]
 [-124 -122 -119 -116 -111 -105 -100  -94  -88  -83]
 [-125 -124 -123 -119 -114 -108 -103  -97  -92  -87]
 [-127 -127 -126 -123 -117 -110 -106 -100  -95  -90]
 [-127 -127 -127 -126 -119 -113 -107 -103  -97  -93]
 [-127 -127 -126 -124 -120 -114 -110 -105 -100  -96]
 [-126 -126 -125 -123 -119 -115 -111 -106 -102  -98]
 [-126 -125 -124 -121 -119 -116 -112 -108 -105 -101]
 [-124 -123 -122 -121 -119 -117 -113 -110 -106 -103]
 [-123 -123 -122 -121 -119 -117 -114 -111 -109 -105]]


In [16]:
import numpy as np
print(st1["elevation"].shape)
st = time.time()
pt1 = [st1["elevation"].values[3,4], st1["elevation"].values[1,2]]
et = time.time()
print('3.1 Get index of point by two subsetting time: ', et-st, 'sec')

st = time.time()
#pt2 = st1["elevation"].values[np.ix_([3,4],[1,2])]
idx_arr = np.array([[3,4],[1,2]])
print(idx_arr)
print(tuple(idx_arr.T))
#https://stackoverflow.com/questions/43034563/numpy-array-of-multiple-indices-replace-with-a-different-matrix
pt2=st1['elevation'].values[tuple(idx_arr.T)]
et = time.time()
print('3.2 Get index of point by numpy ix_ time: ', et-st, 'sec')
print(pt1)
print(pt2)

(10, 10)
3.1 Get index of point by two subsetting time:  0.0030269622802734375 sec
[[3 4]
 [1 2]]
(array([3, 1]), array([4, 2]))
<xarray.Dataset>
Dimensions:    (lon: 10, lat: 10)
Coordinates:
  * lon        (lon) float64 109.2 109.2 109.2 109.2 ... 109.2 109.2 109.2 109.2
  * lat        (lat) float64 10.34 10.34 10.34 10.35 ... 10.36 10.36 10.37 10.37
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(10, 10), meta=np.ndarray>
Attributes: (12/36)
    title:                           The GEBCO_2022 Grid - a continuous terra...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units: 

In [ ]:
#slow
x = xr.DataArray([-45,-44.95833,-44,-43.95822], dims=['longitude'])
y = xr.DataArray([35,35.004167,36,36.004168], dims=['latitude'])
st = time.time()
#ds_s2 = ds.sel(lon=x, lat=y, method='nearest')  
et = time.time()
#print('Exe time: ', et-st, 'sec')
#print(ds_s2)               
#print(ds_s2['elevation'].values)

In [ ]:
#https://stackoverflow.com/questions/58706834/angled-slice-in-xarray
startLat = 35.0
startLon = -45.0
endLat = 36.0
endLon = -43.0
n = 3600/15 #15 arc-second
# not work, need another dimension to project
#x = xr.DataArray(np.linspace(startLon,endLon,n), dims='z')
#y = xr.DataArray(np.linspace(startLat,endLat,n), dims='z')
#st = time.time()
#ds_s3 = ds.interp(lon=x, lat=y, method='linear')
#et = time.time()
#print('Exe time: ', et-st, 'sec')
#print(ds_s3)               


In [ ]:
import orjson
import json
st = time.time()
jt1 = orjson.dumps(xt1.tolist(), option=orjson.OPT_NAIVE_UTC | orjson.OPT_SERIALIZE_NUMPY)
et = time.time()
print('4.1 Convert JSON by orjson time: ', et-st, 'sec')
st = time.time()
jt2 = json.dumps(xt1.tolist())
et = time.time()
print('4.2 Convert JSON by json.dump time: ', et-st, 'sec')

print(jt1)
print(jt2)

In [ ]:
ds.close()

In [ ]:
import zarr
import dask

In [ ]:
#compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
dask.config.set(scheduler='single-threaded')
#da = getattr(ds, 'elevation')
#https://www.programcreek.com/python/example/123571/xarray.decode_cf
#da.attrs['_Unsigned']='false'
ds
ds.to_zarr('GEBCO_2022_sub_ice_topo.zarr', mode='w',
           #encoding={"elevation": {"_Unsigned": "false"}},
           group='gebco') ##compute=False, , encoding={'gebbco': {'compressor': compressor}})


In [17]:
st = time.time()
dz = xr.open_zarr(
    'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
dz

Exe time:  0.004856586456298828 sec


<xarray.Dataset>
Dimensions:    (lat: 43200, lon: 86400)
Coordinates:
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(675, 2700), meta=np.ndarray>
Attributes: (12/36)
    Conventions:                     CF-1.6, ACDD-1.3
    comment:                         The data in the GEBCO_2022 Grid should n...
    creator_email:                   gdacc@seabed2030.org
    creator_name:                    GEBCO through the Nippon Foundation-GEBC...
    creator_type:                    International organisation
    creator_url:                     https://www.gebco.net
    ...                              ...
    node_offset:                     1.0
    project:                         Nippon Foundation - GEBCO Seabed2030 Pro...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    source:                          The GEBCO_2022 Grid is the latest global...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    title:                           The GEBCO_2022 Grid - a continuous terra...

In [ ]:
print(sys.getsizeof(dz))
dz['elevation'].nbytes/(1024 ** 3)

In [ ]:
st = time.time()
dz_s1 = dz.sel(lon=slice(105, 135), lat=slice(2, 35)) 
dz_s1.close() #try if it can close
zt1 = dz_s1.isel(lon=slice(1000, 1010), lat=slice(2000, 2010))
xz1 = zt1['elevation'].values
et = time.time()
print('Exe time: ', et-st, 'sec')
print(xz1)

In [ ]:
st = time.time()
jz1 = orjson.dumps(xz1.tolist(), option=orjson.OPT_NAIVE_UTC | orjson.OPT_SERIALIZE_NUMPY)
et = time.time()
print('4.1z Convert JSON by orjson time: ', et-st, 'sec')
print(jz1)
